In [1]:
import os

dataset_dir = './data/clean'  # Set the dataset directory
train_dir = './data/train'
val_dir = './data/val'

words_path = []

for root, dirs, files in os.walk(dataset_dir, topdown=True):
    for name in files:
        words_path.append(os.path.join(root, name))

In [2]:
import shutil

if os.path.isdir(train_dir):
    shutil.rmtree(train_dir)
os.mkdir(train_dir)
    
if os.path.isdir(val_dir):
    shutil.rmtree(val_dir)
os.mkdir(val_dir)
    
os.mkdir(os.path.join(train_dir, 'isnull'))
os.mkdir(os.path.join(val_dir, 'isnull'))
    
with open('./data/training data dic.txt') as f:
    wordset = f.read().split('\n')

for word in wordset:
    os.mkdir(os.path.join(train_dir, word))
    os.mkdir(os.path.join(val_dir, word))

In [3]:
import random

from PIL import Image

random.shuffle(words_path)

for i in range(len(words_path)):
    img = Image.open(words_path[i])
    if i < len(words_path) * 0.8:
        if words_path[i].split('_')[-1].split('.')[0] in wordset:
            img.save(os.path.join(train_dir, words_path[i].split('_')[-1].split('.')[0], words_path[i].split('/')[-1]))
        else:
            img.save(os.path.join(train_dir, 'isnull', words_path[i].split('/')[-1]))
    else:
        if words_path[i].split('_')[-1].split('.')[0] in wordset:
            img.save(os.path.join(val_dir, words_path[i].split('_')[-1].split('.')[0], words_path[i].split('/')[-1]))
        else:
            img.save(os.path.join(val_dir, 'isnull', words_path[i].split('/')[-1]))